# 1D Heat Eqaution

This example solves Laplace’s equation in one dimension for a certain set of initial conditions and boundary conditions. A full discussion of Laplace’s equation is out of scope for this documentation, but it will suffice to say that it describes how heat propagates through an object over time. It works by discretizing the problem in two ways:

1. The domain is partitioned into a mesh of points that each have an individual temperature.

1. Time is partitioned into discrete intervals that are advanced forward sequentially.

Then, the following assumption is applied: The temperature of a point after some interval has passed is some weighted average of the temperature of the points that are directly adjacent to it. Intuitively, if all the points in the domain are very hot and a single point in the middle is very cold, as time passes, the hot points will cause the cold one to heat up and the cold point will cause the surrounding hot pieces to cool slightly. Simply put, the heat spreads throughout the object.

We can implement this simulation using a Numba kernel. Let’s start simple by assuming we have a one dimensional object which we’ll represent with an array of values. The position of the element in the array is the position of a point within the object, and the value of the element represents the temperature.

In [47]:
import numpy as np
from numba import cuda

Some initial setup here. Let’s make one point in the center of the object very hot.

In [48]:
# Use an odd problem size.
# This is so there can be an element truly in the "middle" for symmetry.
size = 1001
data = np.zeros(size)

# Middle element is made very hot
data[500] = 10000
buf_0 = cuda.to_device(data)

# This extra array is used for synchronization purposes
buf_1 = cuda.device_array_like(buf_0)

niter = 10000

The initial state of the problem can be visualized as:

In [49]:
# plot T vs x using plotly
import plotly.graph_objects as go
fig = go.Figure(data=go.Scatter(x=np.arange(size), y=data))

# label axis Temperature and Position
fig.update_xaxes(title_text='Position')
fig.update_yaxes(title_text='Temperature')

# title the plot 't = 0'
fig.update_layout(title=r't = t_0')
fig.show()

In our kernel each thread will be responsible for managing the temperature update for a single element in a loop over the desired number of timesteps. The kernel is below. Note the use of cooperative group synchronization and the use of two buffers swapped at each iteration to avoid race conditions. See `numba.cuda.cg.this_grid()` for details.

In [50]:
@cuda.jit
def solve_heat_equation(buf_0, buf_1, timesteps, k):
    i = cuda.grid(1)

    # Don't continue if our index is outside the domain
    if i >= len(buf_0):
        return

    # Prepare to do a grid-wide synchronization later
    grid = cuda.cg.this_grid()

    for step in range(timesteps):
        # Select the buffer from the previous timestep
        if (step % 2) == 0:
            data = buf_0
            next_data = buf_1
        else:
            data = buf_1
            next_data = buf_0

        # Get the current temperature associated with this point
        curr_temp = data[i]

        # Apply formula from finite difference equation
        if i == 0:
            # Left wall is held at T = 0
            next_temp = curr_temp + k * (data[i + 1] - (2 * curr_temp))
        elif i == len(data) - 1:
            # Right wall is held at T = 0
            next_temp = curr_temp + k * (data[i - 1] - (2 * curr_temp))
        else:
            # Interior points are a weighted average of their neighbors
            next_temp = curr_temp + k * (
                data[i - 1] - (2 * curr_temp) + data[i + 1]
            )

        # Write new value to the next buffer
        next_data[i] = next_temp

        # Wait for every thread to write before moving on
        grid.sync()

Calling the kernel:

In [51]:
solve_heat_equation.forall(len(data))(
    buf_0, buf_1, niter, 0.25
)
data = buf_1.copy_to_host()

d:\GitHubWSU\pfc-blocks\.venv\Lib\site-packages\numba\cuda\dispatcher.py:536: NumbaPerformanceWarning:

Grid size 2 will likely result in GPU under-utilization due to low occupancy.



Plotting the final data shows an arc that is highest where the object was hot initially and gradually sloping down to zero towards the edges where the temperature is fixed at zero. In the limit of infinite time, the arc will flatten out completely.

In [52]:
# plot T vs x using plotly
import plotly.graph_objects as go
fig = go.Figure(data=go.Scatter(x=np.arange(size), y=data))

# label axis Temperature and Position
fig.update_xaxes(title_text='Position')
fig.update_yaxes(title_text='Temperature')

# title the plot 't = 0'
fig.update_layout(title=f't = {niter}')
fig.show()

## Challenge
Implement using FFT.